Reading instruments SDA 

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime 
import os 
import glob 
import math 

In [24]:
def file_name_dir(data_dir):
    """
    get the raw EC data file names and directories

    input:  the parent directory of the raw EC data
    return: the EC data file names (_name) and directories(_dir)
    
    """    

    cr6_name = os.listdir(data_dir + '\\cr6')
    GPS_name = os.listdir(data_dir + '\\shipdata')
    metek_name = os.listdir(data_dir + '\\metek')
    cr800_name = os.listdir(data_dir + '\\cr800')
    Picarro_name = os.listdir(data_dir + '\\Picarro')

 
    return cr6_name,GPS_name,metek_name,cr800_name,Picarro_name

In [25]:
data_dir = 'C:/Users/ica/OneDrive - Plymouth Marine Laboratory/vscode/EC_co2_flux/EC flux processing/SDA_IMC/SDA_data_sample'

In [16]:
#make new folder to store the processed data
new_folder_path = os.path.join(data_dir, "L0_test")

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
#creates lists of files to be processed 
cr6_name,GPS_name,metek_name,cr800_name,Picarro_name = file_name_dir(data_dir)

GPS data

In [ ]:
'''
function LoadGPS() 
	NewPath /O/Q/M="Choose folder containing ship's GPS data" RawFilePath # file contains lat lon and speed, 

	PathInfo RawFilePath			// Check to see if user created the path.  Returns path in S_Path variable.
	if (V_flag==0)
		return -1					// User cancelled process.
	endif
	
	String FileList = IndexedFile(RawFilePath, -1, ".txt")	

	FileList = SortList(FileList, ";", 0)

	Variable ii, jj
	string tempstr

	for(ii=0;ii<ItemsInList(FileList);ii+=1) #hourly files 

		String RawFile2Load = StringFromList(ii, FileList)							// Get file name from list

		LoadWave/J/D/A/Q/W/K=2/V={"\t, "," $",0,0}/L={0,0,0,0,9}/R={English,2,2,2,2,"Year-Month-DayOfMonth",40}/P=RawFilePath RawFile2Load		#datetime // Load data from file.  Overwrite old variables if they exist.
		// declare loaded waves
		wave/t wave0, wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8 #columns of useful data as string
		Make /o/t/n=(numpnts(wave0)) meas_flag; meas_flag = wave2
		Make /o/t/n=(numpnts(wave0)) lat_flag; lat_flag = wave5
		Make /o/t/n=(numpnts(wave0)) lon_flag; lon_flag = wave7
				
		// reload data as numeric only #
		LoadWave/J/D/A/Q/W/K=1/V={"\t, "," $",0,0}/L={0,0,0,0,9}/R={English,2,2,2,2,"Year-Month-DayOfMonth",40}/P=RawFilePath RawFile2Load		// Load data from file.  Overwrite old variables if they exist.
		Wave wave9, wave10, wave11, wave12, wave13, wave14, wave15, wave16, wave17
		Duplicate /o wave9, timeGPS
						
		Make/d/o/n=(numpnts(wave0)) Lat, Lon, SOG
		Lat = nan
		Lon = nan
		SOG = nan
		

		// pick out lat/lon/SOG data #calculations to convert to lat lons 
		for (jj=0; jj<numpnts(wave0); jj+=1)
		tempstr = meas_flag[jj]
			if (stringmatch(tempstr, "$INGGA")==1) #looking for this specific string 
				lat[jj] = wave13[jj]/100 
				lon[jj] = wave15[jj]/100				
			endif

			if (stringmatch(tempstr, "$INVTG")==1) 
				SOG[jj] = wave16[jj]		// knots #speed over groud 
			endif
		
		endfor 
		
		// convert lat/lon format
		Duplicate /o lat, Lat_deg, Lon_deg		
		Lat_deg = floor(lat)+ (lat - floor(lat))*100/60
		Lon_deg = floor(lon)+ (lon - floor(lon))*100/60

		// add negative sign for south latitude and west longitude
		for (jj=0; jj<numpnts(wave0); jj+=1)
		
		tempstr = lat_flag[jj]
			if (stringmatch(tempstr, "S")==1) # south, add -ve sign 
				Lat_deg[jj] = -Lat_deg[jj]
			endif

		tempstr = lon_flag[jj]
			if (stringmatch(tempstr, "W")==1)
				Lon_deg[jj] = -Lon_deg[jj]
			endif
		endfor 
		
		#prev,minute long files, now seconds 
		Concatenate /NP {timeGPS}, Time_GPS_1min							// Concatnate to results to be saved
		Concatenate /NP {Lat_deg}, lat_1min							// Concatnate to results to be saved
		Concatenate /NP {Lon_deg}, lon_1min								// Concatnate to results to be saved
		Concatenate /NP {SOG}, SOG_1min								// Concatnate to results to be saved

		KillWaves/z wave0, wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8, wave9, wave10
		KillWaves/z wave11, wave12, wave13, wave14, wave15, wave16, wave17, wave18, wave19, time1s
		KillWaves/z wave20, wave21, wave22
		killwaves/z lat, lat_flag, lon, lon_flag, meas_flag, timegps, sog, lat_deg, lon_deg
	endfor

end
'''

In [ ]:
'''
function LoadGyro()
#compass, gyro, whcih way ship is tilting, same as previous file (GPS), gives you the heading 
	NewPath /O/Q/M="Choose folder containing ship's gyro data" RawFilePath
	PathInfo RawFilePath			// Check to see if user created the path.  Returns path in S_Path variable.
	if (V_flag==0)
		return -1					// User cancelled process.
	endif
	
	String FileList = IndexedFile(RawFilePath, -1, ".txt")	

	FileList = SortList(FileList, ";", 0)

	Variable ii, jj
	string tempstr

	for(ii=0;ii<ItemsInList(FileList);ii+=1)

		String RawFile2Load = StringFromList(ii, FileList)							// Get file name from list

		LoadWave/J/D/A/Q/W/K=2/V={"\t, "," $",0,0}/L={0,0,0,0,5}/R={English,2,2,2,2,"Year-Month-DayOfMonth",40}/P=RawFilePath RawFile2Load		// Load data from file.  Overwrite old variables if they exist.
		// declare loaded waves
		wave/t wave0, wave1, wave2, wave3, wave4
		Make /o/t/n=(numpnts(wave0)) meas_flag; meas_flag = wave2
				
		// reload data as numeric only
		LoadWave/J/D/A/Q/W/K=1/V={"\t, "," $",0,0}/L={0,0,0,0,5}/R={English,2,2,2,2,"Year-Month-DayOfMonth",40}/P=RawFilePath RawFile2Load		// Load data from file.  Overwrite old variables if they exist.
		Wave wave5, wave6, wave7, wave8, wave9
		Duplicate /o wave5, timegyro
						
		Make/d/o/n=(numpnts(wave0)) heading
		heading = nan		

		// pick out lat/lon/SOG data
		for (jj=0; jj<numpnts(wave0); jj+=1)
		tempstr = meas_flag[jj]
			if (stringmatch(tempstr, "$INHDT")==1)
				heading[jj] = wave8[jj]
			endif

		endfor 
				
		Concatenate /NP {timegyro}, Time_gyro_1min							// Concatnate to results to be saved
		Concatenate /NP {heading}, heading_1min							// Concatnate to results to be saved

		KillWaves/z wave0, wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8, wave9, wave10
		killwaves/z timegyro, heading, meas_flag
	endfor

end #merge two previous functions 
'''

Wind data 

In [ ]:
for filename  in cr6_name:
    df= pd.read_csv(data_dir + '\\CR6\\' + filename, skiprows=1, delimiter = ',')
    df = df.iloc[2:].reset_index(drop=True)

In [ ]:
def read_cr6_data(df):
    rad2deg = 180.0 / np.pi

    # Extract columns from DataFrame
    x, y, z, t = df["SonicX"], df["SonicY"], df["SonicZ"], df["SonicT"]
    x1, y1, z1, t1 = df["ShipSonicX"], df["ShipSonicY"], df["ShipSonicZ"], df["ShipSonicT"]
    rotx, roty, rotz = df["RotX"], df["RotY"], df["RotZ"]
    accelx, accely, accelz = df["AccX"], df["AccY"], df["AccZ"]

    # Initialize arrays
    u_ms = np.full(len(y), np.nan)
    v_ms = np.full(len(x), np.nan)
    w_ms = np.full(len(z), np.nan)
    t_degC = np.full(len(t), np.nan)
    u_ms_ship = np.full(len(y1), np.nan)
    v_ms_ship = np.full(len(x1), np.nan)
    w_ms_ship = np.full(len(z1), np.nan)
    t_degC_ship = np.full(len(t1), np.nan)

    # Process values
    for j in range(len(x)):
        if x[j] != '':
            v_ms[j] = float(x[j]) / -100.0
            u_ms[j] = float(y[j]) / 100.0
            w_ms[j] = float(z[j]) / 100.0
            t_degC[j] = float(t[j]) / 100.0
            v_ms_ship[j] = float(x1[j]) / -100.0
            u_ms_ship[j] = float(y1[j]) / 100.0
            w_ms_ship[j] = float(z1[j]) / 100.0
            t_degC_ship[j] = float(t1[j]) / 100.0

    # Rotation and Acceleration
    frotx_degs = rad2deg * np.array(rotx, dtype=float) / 1000.0
    froty_degs = rad2deg * np.array(roty, dtype=float) / 1000.0
    frotz_degs = rad2deg * np.array(rotz, dtype=float) / 1000.0
    faccelx_g = np.array(accelx, dtype=float) / -1000.0
    faccely_g = np.array(accely, dtype=float) / -1000.0
    faccelz_g = np.array(accelz, dtype=float) / -1000.0

    return u_ms, v_ms, w_ms, t_degC, u_ms_ship, v_ms_ship, w_ms_ship, t_degC_ship, frotx_degs, froty_degs, frotz_degs, faccelx_g, faccely_g, faccelz_g
